In [66]:
import os
import re
import pandas as pd
import numpy as np
import ssl
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context
import json

import re
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from scipy.special import comb 
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc, confusion_matrix
from sklearn.pipeline import Pipeline
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import wordnet
import string
from sklearn.svm import LinearSVC
nltk.download('popular')
nltk.download('stopwords')

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     /Users/jghosh2/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     /Users/jghosh2/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     /Users/jghosh2/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     /Users/jghosh2/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     /Users/jghosh2/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /Users/jghosh2/nltk_data...
[nltk_data]    |   Package movie_reviews is a

True

In [67]:
df1 = pd.read_csv('/Users/jghosh2/Documents/my-notebook/Tax_ml_poc/data/historical_data/Taxml_Integers_Verified_12-11.csv', index_col=None, header=0,dtype={'item_name': str,'merchant_type_analytics':str},usecols=['item_name','merchant_type_analytics'])
df2 = pd.read_csv('/Users/jghosh2/Documents/my-notebook/Tax_ml_poc/data/historical_data/historical_data_v1.csv', index_col=None, header=0,dtype={'item_name': str, 'description': str,'establishment_type': str, 'target_new':str},usecols=['UniqueUUID','item_name','description','establishment_type','target_new'])


In [89]:
df2['target_new'].value_counts()

CAT_LIQUOR:535                           50774
CAT_TPP:531                              34822
CAT_PREPARED_FOOD,TEMP_HEATED:101,1      34764
CAT_WINE:534                             24863
CAT_BEER:533                             22057
                                         ...  
CAT_PREPACKAGED_FOOD_TOMATO_PUREE:745        3
CAT_PRESCRIPTION_DRUGS:520                   2
CAT_POSTAGE:527                              2
CAT_NEWSPAPERS:538                           1
CAT_OIL:778                                  1
Name: target_new, Length: 178, dtype: int64

In [17]:
df3=df2.merge(df1, on='item_name', how='left')

In [18]:
df3['merchant_type_analytics'].value_counts()

MERCHANT_TYPE_LIQUOR            135061
MERCHANT_TYPE_ESSENTIALS         45314
MERCHANT_TYPE_RETAIL             14395
MERCHANT_TYPE_GROCERY            11756
MERCHANT_TYPE_SPECIALTY_FOOD      6032
MERCHANT_TYPE_PHARMACY            4949
MERCHANT_TYPE_PET_SUPPLY          3447
MERCHANT_TYPE_FLOWER              2400
MERCHANT_TYPE_RESTAURANT          1865
Name: merchant_type_analytics, dtype: int64

In [19]:
df1.shape

(178075, 2)

In [20]:
df2.shape

(381051, 5)

In [21]:
df3.shape

(443592, 6)

In [23]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 443592 entries, 0 to 443591
Data columns (total 6 columns):
 #   Column                   Non-Null Count   Dtype 
---  ------                   --------------   ----- 
 0   UniqueUUID               443592 non-null  object
 1   item_name                443589 non-null  object
 2   description              317632 non-null  object
 3   establishment_type       443592 non-null  object
 4   target_new               443592 non-null  object
 5   merchant_type_analytics  225219 non-null  object
dtypes: object(6)
memory usage: 23.7+ MB


In [25]:
df3['merchant_type_analytics'].isnull().sum()

218373

In [26]:
df3

,UniqueUUID,item_name,description,establishment_type,target_new,merchant_type_analytics
0,\n,#1 Mix: 6 Classic/6 Deluxe Donuts,6 Classics and 6 Deluxe Donuts. Please only re...,BAKERY,"CAT_PREPARED_FOOD,TEMP_HEATED:101,1",MERCHANT_TYPE_SPECIALTY_FOOD
1,\n,#2 Mix: 6 Classic/4 Deluxe/2 Fancy Donuts,"6 Classics, 4 Deluxe and 2 Fancy Donuts. Pleas...",BAKERY,"CAT_PREPARED_FOOD,TEMP_HEATED:101,1",MERCHANT_TYPE_SPECIALTY_FOOD
2,\n,#3 Mix: 6 Classic/4 Deluxe/2 Premium Donuts,"6 Classics, 4 Deluxe and 2 Premium donuts. Ple...",BAKERY,"CAT_PREPARED_FOOD,TEMP_HEATED:101,1",MERCHANT_TYPE_SPECIALTY_FOOD
3,\n,#4 Mix: 3 Classic/3 Deluxe/3 Fancy/3 Premium D...,"3 Classics, 3 Deluxe, 3 Fancy and 3 Premium do...",BAKERY,"CAT_PREPARED_FOOD,TEMP_HEATED:101,1",MERCHANT_TYPE_SPECIALTY_FOOD
4,\n,1. Chicken Chow Mein Lunch Special,"Served with pork fried rice, and either egg dr...",RESTAURANT,"CAT_PREPARED_FOOD,TEMP_HEATED:101,1",MERCHANT_TYPE_LIQUOR
...,...,...,...,...,...,...
443587,\n,Joy · Ice Cream Cups (48 cups),48 cups,GROCERY,"CAT_CANDY,TRAIT_FLOUR,TEMP_COLD:108,61,3",NaN
443588,\n,Fresh Michellada Mix (16 oz),Fresh michellada mix in 4 choices of flavor. U...,CONVENIENCE,"CAT_PREPARED_DRINK,TEMP_UNHEATED:114,2",NaN
443589,\n,Clamato,Clam and tomato juice beverage used to make cl...,CONVENIENCE,"CAT_JUICE,TRAIT_PCT_100,TEMP_COLD:110,51,3",MERCHANT_TYPE_LIQUOR
443590,\n,Clamato,Clam and tomato juice beverage used to make cl...,CONVENIENCE,"CAT_JUICE,TRAIT_PCT_100,TEMP_COLD:110,51,3",MERCHANT_TYPE_LIQUOR


In [80]:
production_data=pd.read_csv('/Users/jghosh2/Documents/my-notebook/Tax_ml_poc/data/production_data/production_data.csv')

In [81]:
production_data.shape

(1048575, 5)

In [82]:
def preprocess_text(message):

    #stopwords
    stpwrd = nltk.corpus.stopwords.words('english')
    lemmatizer = WordNetLemmatizer()
    message=message.lower()
    message = re.sub(r'-',' ', message)
    #removing the numerical values and working only with text values
    message = re.sub('[^a-zA-Z]', " ", message)
    #lowering and removing punctuation
    message = re.sub(r'[^\w\s]',' ',message)
    #removing the stopwords
    message = ' '.join([word for word in message.split() if word not in stpwrd])
    #lemmatizing the text
    message =  " ".join([lemmatizer.lemmatize(w) for w in nltk.word_tokenize(message) if w not in string.punctuation])
    return message

In [90]:
# combine the columns Item, Description and establishment_type into one column 'combined_text'
df2['combined_text'] = df2[['item_name','description','establishment_type']].apply(lambda x: ' '.join(x[x.notnull()]), axis = 1)
# apply data preprocessing steps on the prepared column
df2['processed_text']= df2['combined_text'].map(lambda s:preprocess_text(s)) 

df2 = df2.reset_index(drop=True)
# prepare the target column by combining 'Agent Corrected CAT Name' and 'Agent Corrected Integer'

df7=df2.drop_duplicates(subset=['processed_text','target_new'],keep='first')

In [91]:
df7.shape

(337168, 7)

In [94]:
d=df7.drop(columns='combined_text')

In [95]:
d.to_csv('historical_data_v1.csv')

In [96]:
d.shape

(337168, 6)

In [59]:
df5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 924117 entries, 0 to 924116
Data columns (total 2 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   item_name    924117 non-null  object
 1   description  924117 non-null  object
dtypes: object(2)
memory usage: 14.1+ MB


In [38]:
df5=pd.read_csv('/Users/jghosh2/Documents/my-notebook/Tax_ml_poc/data/production_data/combinedlatestdata.csv',usecols=['item_name','description'])

In [46]:
df6=df4.merge(df5, on='item_name', how='left')

In [47]:
df6

,UniqueUUID,item_name,description_x,establishment_type,target_new,description_y
0,0c1c07dd-71c4-547e-b3c7-7c8e818daff9:3b5ff8bf-...,Thin Crust Pizza (Unbaked),Choose from a variety of unbaked thin crust pi...,CONVENIENCE,"CAT_PREPACKAGED_FOOD,TEMP_COLD:106,3",NaN
1,0c1c07dd-71c4-547e-b3c7-7c8e818daff9:b1372e57-...,Kemp's Heavy Wipping Cream,NaN,CONVENIENCE,"CAT_PREPACKAGED_FOOD,TEMP_COLD:106,3",NaN
2,0c1c07dd-71c4-547e-b3c7-7c8e818daff9:c8e68a97-...,Betty Crocker Yellow Cake Mix,NaN,CONVENIENCE,"CAT_PREPACKAGED_FOOD,TEMP_COLD:106,3",NaN
3,0c1c07dd-71c4-547e-b3c7-7c8e818daff9:96a84d13-...,Oh Snap Pickles,Choose between Hottie Bites and Dilly Bites,CONVENIENCE,"CAT_PREPARED_FOOD,TEMP_HEATED:101,1",NaN
4,0c1c07dd-71c4-547e-b3c7-7c8e818daff9:7b8ffb92-...,Garlic Cloves,NaN,CONVENIENCE,"CAT_PREPACKAGED_FOOD,TEMP_COLD:106,3",NaN
...,...,...,...,...,...,...
1060872,0b0f67f3-de4b-53a4-8bfe-2d2f3b292c3d,Skin republic Â· Hyaluronic acid + collagen fa...,25 mL,GROCERY,CAT_TPP_SKIN_CARE_PRODUCTS:818,NaN
1060873,de6ebc94-88a1-5a74-8743-31d2c057ae14,Garnier Â· Skinactive charcoal peel-off mask (...,"This l i q u i d - t o - p e e l mask,formulat...",GROCERY,CAT_TPP_SKIN_CARE_PRODUCTS:818,NaN
1060874,449994ce-2db8-596a-b6b4-5328d597c63a,Skin republic Â· Foot peel mask (1 unit),1 unit,GROCERY,CAT_TPP_SKIN_CARE_PRODUCTS:818,NaN
1060875,8fa5fab2-6cdc-57c9-a6dd-71472e00c0aa,SkinRepublic Â· Spots & blemish face mask (1 u...,1 u nit,GROCERY,CAT_TPP_SKIN_CARE_PRODUCTS:818,NaN


In [53]:
df7 = df6.loc[df6['description_y'].notnull()]

In [54]:
df7

,UniqueUUID,item_name,description_x,establishment_type,target_new,description_y
11,0c1c07dd-71c4-547e-b3c7-7c8e818daff9:7c71bf50-...,Lettuce Head,NaN,CONVENIENCE,"CAT_PREPACKAGED_FOOD,TEMP_COLD:106,3",Fresh Produce Fresh head of lettuce
12,0c1c07dd-71c4-547e-b3c7-7c8e818daff9:7c71bf50-...,Lettuce Head,NaN,CONVENIENCE,"CAT_PREPACKAGED_FOOD,TEMP_COLD:106,3",Fresh Produce Fresh head of lettuce
15,0c1c07dd-71c4-547e-b3c7-7c8e818daff9:9f0c1df6-...,Skittles,Choose between Regular and King Size,CONVENIENCE,"CAT_CANDY,TEMP_COLD:108,3",Candy
16,0c1c07dd-71c4-547e-b3c7-7c8e818daff9:9f0c1df6-...,Skittles,Choose between Regular and King Size,CONVENIENCE,"CAT_CANDY,TEMP_COLD:108,3",Chocolates Candies \N
17,0c1c07dd-71c4-547e-b3c7-7c8e818daff9:9f0c1df6-...,Skittles,Choose between Regular and King Size,CONVENIENCE,"CAT_CANDY,TEMP_COLD:108,3",Beverages
...,...,...,...,...,...,...
988552,b7719a0e-1d65-5251-ad99-bfc4f6447b94,Pumpkin,1 unit (approx. 2 kg),GROCERY,CAT_PREPACKAGED_FOOD_FRESH_FRUITS:752,Freshly Brewed Iced Coffee Contains: Pumpkin s...
988553,b7719a0e-1d65-5251-ad99-bfc4f6447b94,Pumpkin,1 unit (approx. 2 kg),GROCERY,CAT_PREPACKAGED_FOOD_FRESH_FRUITS:752,"Chai Teas Contains: Chai Base, Pumpkin sauce (..."
988562,39fb9298-8040-5631-8a26-b0cf42c6d59d,Pumpkin,1 tray (approx. 200 g),GROCERY,CAT_PREPACKAGED_FOOD_FRESH_FRUITS:752,Milkshakes Contains: Pumpkin sauce (1|2)
988563,39fb9298-8040-5631-8a26-b0cf42c6d59d,Pumpkin,1 tray (approx. 200 g),GROCERY,CAT_PREPACKAGED_FOOD_FRESH_FRUITS:752,Freshly Brewed Iced Coffee Contains: Pumpkin s...
